In [1]:
import os
import xml.etree.ElementTree as ET
import shutil
import random
from tqdm import tqdm  # For progress bars
import yaml

In [2]:
base_path = 'archive/train'

# Output path for the subset dataset
output_base_path = '/working/rdd2020_subset_yolo'

# List of countries
countries = ['Czech', 'Japan', 'India']

# Percentage of data to use
subset_ratio = 1  # 100%

In [3]:
for split in ['train', 'val']:
    os.makedirs(os.path.join(output_base_path, 'images', split), exist_ok=True)
    os.makedirs(os.path.join(output_base_path, 'labels', split), exist_ok=True)

class_name_to_id = {}
split_ratio = 0.8  

for country in countries:
    annotations_xmls_path = os.path.join(base_path, country, 'annotations', 'xmls')
    images_path = os.path.join(base_path, country, 'images')

    xml_files = [f for f in os.listdir(annotations_xmls_path) if f.lower().endswith('.xml')]
    num_subset_files = max(1, int(len(xml_files) * subset_ratio)) 

    subset_xml_files = random.sample(xml_files, num_subset_files)

    annotated_images = []
    for xml_file in subset_xml_files:
        ann_path = os.path.join(annotations_xmls_path, xml_file)
        try:
            tree = ET.parse(ann_path)
            root = tree.getroot()
            filename = root.find('filename').text
            if filename is None:
                filename = os.path.splitext(xml_file)[0] + '.jpg'
            annotated_images.append((filename, xml_file))
        except ET.ParseError as e:
            print(f"Error parsing {ann_path}: {e}")
            continue

    random.shuffle(annotated_images)
    split_index = int(len(annotated_images) * split_ratio)
    train_images = annotated_images[:split_index]
    val_images = annotated_images[split_index:]

    for split, image_list in zip(['train', 'val'], [train_images, val_images]):
        for image_file, xml_file in tqdm(image_list, desc=f"Processing {country} {split} set"):
            # Paths
            image_src_path = os.path.join(images_path, image_file)
            image_dst_path = os.path.join(output_base_path, 'images', split, image_file)
            label_dst_path = os.path.join(output_base_path, 'labels', split, f"{os.path.splitext(image_file)[0]}.txt")
            ann_path = os.path.join(annotations_xmls_path, xml_file)

            if not os.path.exists(image_src_path):
                print(f"Image file {image_src_path} does not exist. Skipping.")
                continue

            shutil.copy(image_src_path, image_dst_path)

            try:
                tree = ET.parse(ann_path)
                root = tree.getroot()
            except ET.ParseError as e:
                print(f"Error parsing {ann_path}: {e}")
                continue

            size = root.find('size')
            if size is not None:
                width = int(size.find('width').text)
                height = int(size.find('height').text)
            else:
                print(f"Size missing in annotation {ann_path}. Skipping.")
                continue

            with open(label_dst_path, 'w') as label_file:
                for obj in root.findall('object'):
                    class_name = obj.find('name').text
                    if class_name is None:
                        continue

                    if class_name not in class_name_to_id:
                        class_id = len(class_name_to_id)
                        class_name_to_id[class_name] = class_id
                    else:
                        class_id = class_name_to_id[class_name]

                    bndbox = obj.find('bndbox')
                    if bndbox is not None:
                        xmin = float(bndbox.find('xmin').text)
                        ymin = float(bndbox.find('ymin').text)
                        xmax = float(bndbox.find('xmax').text)
                        ymax = float(bndbox.find('ymax').text)
                    else:
                        continue

                    x_center = ((xmin + xmax) / 2) / width
                    y_center = ((ymin + ymax) / 2) / height
                    bbox_width = (xmax - xmin) / width
                    bbox_height = (ymax - ymin) / height

                    label_file.write(f"{class_id} {x_center} {y_center} {bbox_width} {bbox_height}\n")

data = {
    'train': os.path.join(output_base_path, 'images', 'train'),
    'val': os.path.join(output_base_path, 'images', 'val'),
    'nc': len(class_name_to_id),
    'names': list(class_name_to_id.keys())
}

with open(os.path.join(output_base_path, 'data.yaml'), 'w') as f:
    yaml.dump(data, f, default_flow_style=False)

print("Subset creation completed successfully!")
print(f"Number of classes: {len(class_name_to_id)}")
print("Class mappings:")
for class_name, class_id in class_name_to_id.items():
    print(f"{class_id}: {class_name}")

Processing India val set: 100%|██████████| 1542/1542 [00:18<00:00, 85.37it/s] 

Subset creation completed successfully!
Number of classes: 10
Class mappings:
0: D40
1: D00
2: D10
3: D20
4: D50
5: D43
6: D44
7: D01
8: D11
9: D0w0


In [4]:
data = {
    'train': os.path.join(output_base_path, 'images', 'train'),
    'val': os.path.join(output_base_path, 'images', 'val'),
    'nc': len(class_name_to_id),
    'names': list(class_name_to_id.keys())
}

with open(os.path.join(output_base_path, 'data.yaml'), 'w') as f:
    yaml.dump(data, f, default_flow_style=False)

print("Subset creation completed successfully!")
print(f"Number of classes: {len(class_name_to_id)}")
print("Class mappings:")
for class_name, class_id in class_name_to_id.items():
    print(f"{class_id}: {class_name}")

Subset creation completed successfully!
Number of classes: 10
Class mappings:
0: D40
1: D00
2: D10
3: D20
4: D50
5: D43
6: D44
7: D01
8: D11
9: D0w0


In [6]:
data_yaml = os.path.join(output_base_path, 'data.yaml')

# Choose the YOLOv8 model version
model_type = 'yolov8s.pt'  # 'yolov8n.pt' is the Nano version, suitable for quick testing

# Number of epochs for training
epochs = 20 

# Set image size
img_size = 640 

In [7]:
model = YOLO(model_type)


100%|██████████| 21.5M/21.5M [00:13<00:00, 1.68MB/s]


In [10]:
import wandb
wandb.disabled = True

import os
os.environ['WANDB_MODE'] = 'disabled'

# Train the model
model.train(data=data_yaml, epochs=epochs, imgsz=img_size, device=0)

New https://pypi.org/project/ultralytics/8.3.199 available  Update with 'pip install -U ultralytics'
Ultralytics 8.3.135  Python-3.9.19 torch-1.12.1 CUDA:0 (NVIDIA GeForce RTX 4060 Laptop GPU, 8188MiB)
WARNING Upgrade to torch>=2.0.0 for deterministic training.
engine\trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=16, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=/working/rdd2020_subset_yolo\data.yaml, degrees=0.0, deterministic=True, device=0, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=20, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=640, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolov8s.pt, momentum=0.937, mos

100%|██████████| 5.35M/5.35M [00:01<00:00, 3.35MB/s]


AMP: checks passed 
train: Fast image access  (ping: 0.10.1 ms, read: 3.92.3 MB/s, size: 61.0 KB)


train: Scanning C:\working\rdd2020_subset_yolo\labels\train... 16831 images, 5167 backgrounds, 0 corrupt: 100%|██████████| 16831/16831 [00:49<00:00, 339.35it/s]

train: C:\working\rdd2020_subset_yolo\images\train\Japan_006916.jpg: 1 duplicate labels removed


...

                   all       4210       6890      0.523      0.585       0.55       0.29



20 epochs completed in 2.494 hours.
Optimizer stripped from runs\detect\train\weights\last.pt, 22.5MB
Optimizer stripped from runs\detect\train\weights\best.pt, 22.5MB

Validating runs\detect\train\weights\best.pt...
Ultralytics 8.3.135  Python-3.9.19 torch-1.12.1 CUDA:0 (NVIDIA GeForce RTX 4060 Laptop GPU, 8188MiB)
Model summary (fused): 72 layers, 11,129,454 parameters, 0 gradients, 28.5 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 132/132 [00:44<00:00,  2.96it/s]


                   all       4210       6890      0.597      0.485      0.541      0.297
                   D40        621       1142      0.612      0.402      0.477      0.206
                   D00        949       1343      0.551      0.369      0.417      0.191
                   D10        512        863      0.536       0.39      0.422      0.168
                   D20       1307       1673      0.662      0.575      0.656      0.337
                   D50        540        721      0.835        0.8      0.868      0.486
                   D43        145        155      0.713      0.755      0.794      0.515
                   D44        789        966      0.585      0.709      0.677      0.421
                   D01         14         24      0.415     0.0346      0.114     0.0429
                   D11          3          3      0.461      0.333      0.446      0.305
Speed: 0.3ms preprocess, 6.2ms inference, 0.0ms loss, 1.2ms postprocess per image
Results saved to runs\detect

ultralytics.utils.metrics.DetMetrics object with attributes:

ap_class_index: array([0, 1, 2, 3, 4, 5, 6, 7, 8])
box: ultralytics.utils.metrics.Metric object
confusion_matrix: <ultralytics.utils.metrics.ConfusionMatrix object at 0x000002BCAE5041C0>
curves: ['Precision-Recall(B)', 'F1-Confidence(B)', 'Precision-Confidence(B)', 'Recall-Confidence(B)']
curves_results: [[array([          0,    0.001001,    0.002002,    0.003003,    0.004004,    0.005005,    0.006006,    0.007007,    0.008008,    0.009009,     0.01001,    0.011011,    0.012012,    0.013013,    0.014014,    0.015015,    0.016016,    0.017017,    0.018018,    0.019019,     0.02002,    0.021021,    0.022022,    0.023023,
          0.024024,    0.025025,    0.026026,    0.027027,    0.028028,    0.029029,     0.03003,    0.031031,    0.032032,    0.033033,    0.034034,    0.035035,    0.036036,    0.037037,    0.038038,    0.039039,     0.04004,    0.041041,    0.042042,    0.043043,    0.044044,    0.045045,    0.046046,    0.

In [9]:
pip install wandb

  Using cached click-8.1.8-py3-none-any.whl.metadata (2.3 kB)
  Using cached eval_type_backport-0.2.2-py3-none-any.whl.metadata (2.2 kB)
     ---------------------------------------- 0.0/68.4 kB ? eta -:--:--
     ----------------- -------------------- 30.7/68.4 kB 660.6 kB/s eta 0:00:01
     -------------------------------------- 68.4/68.4 kB 746.2 kB/s eta 0:00:00
  Using cached gitdb-4.0.12-py3-none-any.whl.metadata (1.2 kB)
  Using cached annotated_types-0.7.0-py3-none-any.whl.metadata (15 kB)
  Using cached pydantic_core-2.33.2-cp39-cp39-win_amd64.whl.metadata (6.9 kB)
  Using cached typing_inspection-0.4.1-py3-none-any.whl.metadata (2.6 kB)
  Using cached smmap-5.0.2-py3-none-any.whl.metadata (4.3 kB)
   ---------------------------------------- 0.0/18.7 MB ? eta -:--:--
   ---------------------------------------- 0.1/18.7 MB 5.1 MB/s eta 0:00:04
    --------------------------------------- 0.2/18.7 MB 3.0 MB/s eta 0:00:07
    --------------------------------------- 0.4/18.7 MB 3.4

In [11]:
metrics = model.val()


Ultralytics 8.3.135  Python-3.9.19 torch-1.12.1 CUDA:0 (NVIDIA GeForce RTX 4060 Laptop GPU, 8188MiB)
Model summary (fused): 72 layers, 11,129,454 parameters, 0 gradients, 28.5 GFLOPs
val: Fast image access  (ping: 0.10.0 ms, read: 725.8290.2 MB/s, size: 66.8 KB)


val: Scanning C:\working\rdd2020_subset_yolo\labels\val.cache... 4210 images, 1305 backgrounds, 0 corrupt: 100%|██████████| 4210/4210 [00:00<?, ?it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 264/264 [00:50<00:00,  5.26it/s]


                   all       4210       6890      0.599      0.484      0.541      0.297
                   D40        621       1142      0.615      0.399      0.476      0.207
                   D00        949       1343      0.551      0.365      0.417      0.191
                   D10        512        863      0.544      0.389      0.425      0.168
                   D20       1307       1673      0.664      0.574      0.656      0.337
                   D50        540        721      0.836      0.799      0.869      0.483
                   D43        145        155      0.723      0.755      0.793      0.516
                   D44        789        966      0.586      0.707      0.677      0.422
                   D01         14         24      0.409     0.0341      0.114     0.0429
                   D11          3          3      0.462      0.333      0.446      0.305
Speed: 0.3ms preprocess, 7.3ms inference, 0.0ms loss, 1.2ms postprocess per image
Results saved to runs\detect

In [12]:
print(f"Validation mAP50: {metrics.box.map50:.4f}")
print(f"Validation mAP50-95: {metrics.box.map:.4f}")

Validation mAP50: 0.5414
Validation mAP50-95: 0.2969


In [15]:
save_directory = ''
save_filename = 'best_model.pt'  
save_path = os.path.join(save_directory, save_filename)

model.save(save_path)

print(f"Model saved successfully at {save_path}")

Model saved successfully at best_model.pt


In [1]:
from ultralytics import YOLO
m = YOLO('best_model.pt')
with open('labels.txt','w') as f:
    for i in range(len(m.names)):
        f.write(m.names[i] + '\n')
print(m.names)


WARNING Ultralytics settings reset to default values. This may be due to a possible problem with your settings or a recent ultralytics package update. 
View Ultralytics Settings with 'yolo settings' or at 'C:\Users\er_po\AppData\Roaming\Ultralytics\settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.
{0: 'D40', 1: 'D00', 2: 'D10', 3: 'D20', 4: 'D50', 5: 'D43', 6: 'D44', 7: 'D01', 8: 'D11', 9: 'D0w0'}


In [2]:
from ultralytics import YOLO
model = YOLO("best_model.pt")                     # kendi eğittiğin model
model.export(format="tflite", imgsz=640,    # 640'ı eğitimde kullandığınla eşleştir
             half=True,                     # FP16 (daha küçük / hızlı)
             # int8=True, data="data.yaml", # INT8 istiyorsan temsilci veri gerekir
             nms=True)                      # TFLite grafiğine NMS ekle (isteğe bağlı)
# Çıktı ör: best_float16.tflite


Ultralytics 8.3.200  Python-3.10.18 torch-2.8.0+cpu CPU (13th Gen Intel Core(TM) i7-13700H)

PyTorch: starting from 'best_model.pt' with input shape (1, 3, 640, 640) BCHW and output shape(s) (1, 300, 6) (21.4 MB)
ERROR TensorFlow SavedModel: export failure 0.2s: DLL load failed while importing _pywrap_tfe: Belirtilen yordam bulunamad.


ImportError: DLL load failed while importing _pywrap_tfe: Belirtilen yordam bulunamadı.